In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
tr_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [6]:
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform=ToTensor(),
)

In [8]:
batch_size = 64

#create data loaders
tr_dataloader =  DataLoader(tr_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for x,y in test_dataloader:
    print(f"shape of x[N,C,H,W]: {x.shape}")
    print(f"shape of y: {y.shape} {y.dtype}")
    break

shape of x[N,C,H,W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64


In [9]:
# Get CPU, GPU or MPS device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")



Using cpu device


In [10]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flattern = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,xx):
        xx = self.flattern(xx)
        logits = self.linear_relu_stack(xx)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flattern): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
# Optimizing the model parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        # Compute prediction error
        pred = model(x)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [22]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------------------")
    train(tr_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------------------------
loss: 2.182451  [   64/60000]
loss: 2.173256  [ 6464/60000]
loss: 2.124861  [12864/60000]
loss: 2.137191  [19264/60000]
loss: 2.079765  [25664/60000]
loss: 2.040083  [32064/60000]
loss: 2.055743  [38464/60000]
loss: 1.989784  [44864/60000]
loss: 1.993903  [51264/60000]
loss: 1.917793  [57664/60000]
Test Error: 
 Accuracy: 57.4%, Avg loss: 1.920681 

Epoch 2
----------------------------------------------
loss: 1.950999  [   64/60000]
loss: 1.920612  [ 6464/60000]
loss: 1.813970  [12864/60000]
loss: 1.848483  [19264/60000]
loss: 1.728804  [25664/60000]
loss: 1.692943  [32064/60000]
loss: 1.705989  [38464/60000]
loss: 1.612233  [44864/60000]
loss: 1.635736  [51264/60000]
loss: 1.521456  [57664/60000]
Test Error: 
 Accuracy: 58.8%, Avg loss: 1.544791 

Epoch 3
----------------------------------------------
loss: 1.611457  [   64/60000]
loss: 1.570752  [ 6464/60000]
loss: 1.429320  [12864/60000]
loss: 1.492967  [19264/60000]
loss: 1.364

In [24]:
# Saving Models
torch.save(model.state_dict(), "model.pth")
print("saved Pytorch Model State to model.pth")

saved Pytorch Model State to model.pth


In [25]:
# Loading Models

model =  NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [29]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
